Horizon 1 Transformer

In [2]:
import pandas as pd
import numpy as np
import time
import os
import psutil
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras import layers, models
import tensorflow as tf
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft

np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Load and Preprocess Data
# ============================================

def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath)
    df.columns = df.columns.str.replace('"', '').str.strip()
    if 'TIME' not in df.columns:
        raise KeyError("Column 'TIME' missing.")
    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)
    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        target_col = 'target'
    if target_col not in df.columns:
        raise KeyError("Target column not found.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col].astype(int)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)
    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    df['month'] = df.index.month
    df['weekofyear'] = df.index.isocalendar().week
    df['year'] = df.index.isocalendar().year
    df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    fft_values = fft(df[target_col].dropna().values)
    fft_real = np.real(fft_values)[:len(df[target_col])]
    fft_imag = np.imag(fft_values)[:len(df[target_col])]
    df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col].astype(int)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # 3D for Transformer
    return X_scaled, y, scaler

# ============================================
# 3. Transformer Model
# ============================================
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = models.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def build_transformer_model(input_shape, num_classes=4, embed_dim=64, num_heads=2, ff_dim=128, dropout_rate=0.1):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Dense(embed_dim)(inputs)
    x = TransformerBlock(embed_dim, num_heads, ff_dim, dropout_rate)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# ============================================
# 4. Objective Function
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_transformer_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        embed_dim=int(num_units),
        num_heads=2,
        ff_dim=int(num_units * 2),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (30, 60),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train and Evaluate
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_transformer_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        embed_dim=int(best_params['num_units']),
        num_heads=2,
        ff_dim=int(best_params['num_units'] * 2),
        dropout_rate=float(best_params['dropout_rate'])
    )

    # Training time
    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    # Testing time
    start_test = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    total_testing_time = time.time() - start_test
    inference_time_per_sample = total_testing_time / len(X_test)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, digits=4))
    print(confusion_matrix(y_test, y_pred))

    model.save("temp_model.h5")
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Total Testing Time: {total_testing_time:.2f}s")
    print(f"Inference Time per Sample: {inference_time_per_sample:.6f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main Execution
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Hyperparameters:", best_params)

    print("\n Training Final Transformer Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------

| 1         | 1.0       | 33.98     | 0.4803    | 51.96     | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 31.74     | 115.2     |
| 3         | 1.0       | 44.85     | 0.3832    | 30.62     | 125.1     |
| 4         | 1.0       | 55.96     | 0.1849    | 35.45     | 49.61     |
| 5         | 1.0       | 30.6      | 0.3099    | 42.96     | 59.96     |
| 6         | 1.0       | 18.1      | 0.1947    | 56.69     | 32.88     |
| 7         | 1.0       | 63.83     | 0.1911    | 59.46     | 127.8     |
| 8         | 1.0       | 63.87     | 0.1809    | 55.16     | 32.23     |
| 9         | 1.0       | 16.41     | 0.4657    | 59.32     | 127.3     |
| 10        | 0.9959    | 17.67     | 0.4729    | 30.03     | 32.79     |
| 11        | 1.0       | 16.37     | 0.4635    | 56.61     | 33.91     |
| 12 


===== Evaluation =====
Accuracy: 0.9373
              precision    recall  f1-score   support

           0     1.0000    0.7679    0.8687        56
           1     0.9478    0.9833    0.9652       240
           2     0.9912    0.9302    0.9597       487
           3     0.4688    1.0000    0.6383        30

    accuracy                         0.9373       813
   macro avg     0.8519    0.9203    0.8580       813
weighted avg     0.9597    0.9373    0.9432       813

[[ 43  13   0   0]
 [  0 236   4   0]
 [  0   0 453  34]
 [  0   0   0  30]]

===== Model Metrics =====
Training Time: 24.07s
Total Testing Time: 0.56s
Inference Time per Sample: 0.000693s
Model Size: 1.39 MB
Trainable Params: 115,615
RAM Usage: 1435.55 MB


Horizon 2 Transformer

In [2]:
import pandas as pd
import numpy as np
import time
import os
import psutil
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras import layers, models
import tensorflow as tf
from bayes_opt import BayesianOptimization

np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Load and Preprocess Data
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath)
    df.columns = df.columns.str.replace('"', '').str.strip()
    if 'TIME' not in df.columns:
        raise KeyError("Column 'TIME' missing.")
    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)
    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h2'
    if target_col not in df.columns:
        target_col = 'target'
    if target_col not in df.columns:
        raise KeyError("Target column not found.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col].astype(int)
    df[target_col] = df[target_col] - 1  # ensure 0-indexed

    if df[target_col].min() < 0:
        raise ValueError("Invalid label: contains negative class.")

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col].astype(int)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # 3D for Transformer
    return X_scaled, y, scaler

# ============================================
# 3. Transformer Model
# ============================================
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = models.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def build_transformer_model(input_shape, num_classes=4, embed_dim=64, num_heads=2, ff_dim=128, dropout_rate=0.1):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Dense(embed_dim)(inputs)
    x = TransformerBlock(embed_dim, num_heads, ff_dim, dropout_rate)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# ============================================
# 4. Objective Function
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_transformer_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        embed_dim=int(num_units),
        num_heads=2,
        ff_dim=int(num_units * 2),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (30, 60),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train and Evaluate
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_transformer_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        embed_dim=int(best_params['num_units']),
        num_heads=2,
        ff_dim=int(best_params['num_units'] * 2),
        dropout_rate=float(best_params['dropout_rate'])
    )

    # Training time
    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    # Testing time
    start_test = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    total_testing_time = time.time() - start_test
    inference_time_per_sample = total_testing_time / len(X_test)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, digits=4))
    print(confusion_matrix(y_test, y_pred))

    model.save("temp_model.h5")
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Total Testing Time: {total_testing_time:.2f}s")
    print(f"Inference Time per Sample: {inference_time_per_sample:.6f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main Execution
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\train_lasso_selected.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\Lasso_Selected_Features_H2\test_lasso_selected.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Hyperparameters:", best_params)

    print("\n Training Final Transformer Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------



| 1         | 1.0       | 33.98     | 0.4803    | 51.96     | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 31.74     | 115.2     |
| 3         | 1.0       | 44.85     | 0.3832    | 30.62     | 125.1     |
| 4         | 1.0       | 55.96     | 0.1849    | 35.45     | 49.61     |
| 5         | 1.0       | 30.6      | 0.3099    | 42.96     | 59.96     |
| 6         | 1.0       | 18.1      | 0.1947    | 56.69     | 32.88     |
| 7         | 1.0       | 63.83     | 0.1911    | 59.46     | 127.8     |
| 8         | 1.0       | 63.87     | 0.1809    | 55.16     | 32.23     |
| 9         | 1.0       | 16.41     | 0.4657    | 59.32     | 127.3     |
| 10        | 0.9959    | 17.67     | 0.4729    | 30.03     | 32.79     |
| 11        | 1.0       | 16.37     | 0.4635    | 56.61     | 33.91     |
| 12        | 1.0       | 49.69     | 0.1556    | 59.54     | 59.68     |
| 13        | 1.0       | 52.97     | 0.1097    | 31.86     | 78.41     |
| 14        | 1.0       | 63.5      | 


===== Evaluation =====
Accuracy: 0.9828
              precision    recall  f1-score   support

           0     0.9118    0.7381    0.8158        42
           1     0.9530    0.9867    0.9696       226
           2     1.0000    1.0000    1.0000       509
           3     1.0000    1.0000    1.0000        36

    accuracy                         0.9828       813
   macro avg     0.9662    0.9312    0.9463       813
weighted avg     0.9824    0.9828    0.9820       813

[[ 31  11   0   0]
 [  3 223   0   0]
 [  0   0 509   0]
 [  0   0   0  36]]

===== Model Metrics =====
Training Time: 35.93s
Total Testing Time: 1.18s
Inference Time per Sample: 0.001453s
Model Size: 1.37 MB
Trainable Params: 113,746
RAM Usage: 1678.22 MB


Horizon 3 Transformer

In [3]:
import pandas as pd
import numpy as np
import time
import os
import psutil
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras import layers, models
import tensorflow as tf
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft

np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Load and Preprocess Data
# ============================================

def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath)
    df.columns = df.columns.str.replace('"', '').str.strip()
    if 'TIME' not in df.columns:
        raise KeyError("Column 'TIME' missing.")
    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)
    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h3'
    if target_col not in df.columns:
        target_col = 'target'
    if target_col not in df.columns:
        raise KeyError("Target column not found.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col].astype(int)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)
    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()



    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col].astype(int)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # 3D for Transformer
    return X_scaled, y, scaler

# ============================================
# 3. Transformer Model
# ============================================
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = models.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def build_transformer_model(input_shape, num_classes=4, embed_dim=64, num_heads=2, ff_dim=128, dropout_rate=0.1):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Dense(embed_dim)(inputs)
    x = TransformerBlock(embed_dim, num_heads, ff_dim, dropout_rate)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# ============================================
# 4. Objective Function
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_transformer_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        embed_dim=int(num_units),
        num_heads=2,
        ff_dim=int(num_units * 2),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (30, 60),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train and Evaluate
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_transformer_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        embed_dim=int(best_params['num_units']),
        num_heads=2,
        ff_dim=int(best_params['num_units'] * 2),
        dropout_rate=float(best_params['dropout_rate'])
    )

    # Training time
    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    # Testing time
    start_test = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    total_testing_time = time.time() - start_test
    inference_time_per_sample = total_testing_time / len(X_test)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, digits=4))
    print(confusion_matrix(y_test, y_pred))

    model.save("temp_model.h5")
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Total Testing Time: {total_testing_time:.2f}s")
    print(f"Inference Time per Sample: {inference_time_per_sample:.6f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main Execution
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H3_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H3_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Hyperparameters:", best_params)

    print("\n Training Final Transformer Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 1.0       | 33.98     | 0.4803    | 51.96     | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 31.74     | 115.2     |
| 3         | 1.0       | 44.85     | 0.3832    | 30.62     | 125.1     |
| 4         | 0.9959    | 55.96     | 0.1849    | 35.45     | 49.61     |
| 5         | 0.9959    | 30.6      | 0.3099    | 42.96     | 59.96     |
| 6         | 1.0       | 33.45     | 0.2494    | 59.8      | 127.5     |
| 7         | 1.0       | 63.58     | 0.3142    | 59.66     | 103.7     |
| 8         | 0.9959    | 24.08     | 0.3354    | 32.56     | 116.3     |
| 9         | 1.0       | 62.6      | 0.3838    | 59.82     | 67.9      |
| 10        | 0.9959    | 38.63     | 0.1642    | 54.06     | 41.26     |
| 11        | 0.9959    | 61.04     | 0.3825    | 43.5      | 127.9     |
| 12  


===== Evaluation =====
Accuracy: 0.9705
              precision    recall  f1-score   support

           0     0.8333    0.7407    0.7843        27
           1     0.9714    0.9577    0.9645       213
           2     0.9923    0.9847    0.9885       524
           3     0.8305    1.0000    0.9074        49

    accuracy                         0.9705       813
   macro avg     0.9069    0.9208    0.9112       813
weighted avg     0.9718    0.9705    0.9706       813

[[ 20   6   0   1]
 [  4 204   4   1]
 [  0   0 516   8]
 [  0   0   0  49]]

===== Model Metrics =====
Training Time: 21.52s
Total Testing Time: 0.53s
Inference Time per Sample: 0.000646s
Model Size: 1.38 MB
Trainable Params: 114,636
RAM Usage: 2203.01 MB


Horizon 4 Transformer

In [4]:
import pandas as pd
import numpy as np
import time
import os
import psutil
import warnings
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras import layers, models
import tensorflow as tf
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft

np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Load and Preprocess Data
# ============================================

def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath)
    df.columns = df.columns.str.replace('"', '').str.strip()
    if 'TIME' not in df.columns:
        raise KeyError("Column 'TIME' missing.")
    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)
    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h4'
    if target_col not in df.columns:
        target_col = 'target'
    if target_col not in df.columns:
        raise KeyError("Target column not found.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col].astype(int)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)
    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()



    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col].astype(int)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # 3D for Transformer
    return X_scaled, y, scaler

# ============================================
# 3. Transformer Model
# ============================================
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = models.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def build_transformer_model(input_shape, num_classes=4, embed_dim=64, num_heads=2, ff_dim=128, dropout_rate=0.1):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Dense(embed_dim)(inputs)
    x = TransformerBlock(embed_dim, num_heads, ff_dim, dropout_rate)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

# ============================================
# 4. Objective Function
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_transformer_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        embed_dim=int(num_units),
        num_heads=2,
        ff_dim=int(num_units * 2),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (30, 60),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train and Evaluate
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_transformer_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        embed_dim=int(best_params['num_units']),
        num_heads=2,
        ff_dim=int(best_params['num_units'] * 2),
        dropout_rate=float(best_params['dropout_rate'])
    )

    # Training time
    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    # Testing time
    start_test = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    total_testing_time = time.time() - start_test
    inference_time_per_sample = total_testing_time / len(X_test)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred, digits=4))
    print(confusion_matrix(y_test, y_pred))

    model.save("temp_model.h5")
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Total Testing Time: {total_testing_time:.2f}s")
    print(f"Inference Time per Sample: {inference_time_per_sample:.6f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main Execution
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H4_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H4_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Hyperparameters:", best_params)

    print("\n Training Final Transformer Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.9959    | 33.98     | 0.4803    | 51.96     | 89.47     |
| 2         | 0.9959    | 23.49     | 0.1624    | 31.74     | 115.2     |
| 3         | 0.9959    | 44.85     | 0.3832    | 30.62     | 125.1     |
| 4         | 0.9919    | 55.96     | 0.1849    | 35.45     | 49.61     |
| 5         | 0.9959    | 30.6      | 0.3099    | 42.96     | 59.96     |
| 6         | 0.9959    | 35.47     | 0.1346    | 30.11     | 118.2     |
| 7         | 0.9959    | 16.22     | 0.1457    | 31.66     | 78.0      |
| 8         | 0.9959    | 23.79     | 0.3341    | 30.08     | 116.1     |
| 9         | 0.9959    | 16.5      | 0.4436    | 59.54     | 54.2      |
| 10        | 0.9959    | 62.47     | 0.3844    | 59.61     | 127.1     |
| 11        | 0.9959    | 18.27     | 0.3674    | 59.63     | 128.0     |
| 12  


===== Evaluation =====
Accuracy: 0.9422
              precision    recall  f1-score   support

           0     0.5000    0.0769    0.1333        13
           1     0.9392    0.8543    0.8947       199
           2     0.9763    0.9889    0.9826       541
           3     0.7317    1.0000    0.8451        60

    accuracy                         0.9422       813
   macro avg     0.7868    0.7300    0.7139       813
weighted avg     0.9415    0.9422    0.9373       813

[[  1  11   0   1]
 [  1 170  13  15]
 [  0   0 535   6]
 [  0   0   0  60]]

===== Model Metrics =====
Training Time: 20.38s
Total Testing Time: 0.52s
Inference Time per Sample: 0.000636s
Model Size: 1.38 MB
Trainable Params: 114,636
RAM Usage: 2797.50 MB
